In [1]:
from fastai.vision import *

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'1.13.1'

In [4]:
import tensorflow.keras as keras

In [5]:
keras.__version__

'2.2.4-tf'

In [6]:
keras.applications.ResNet50()

Instructions for updating:
Colocations handled automatically by placer.


### Paths

In [7]:
MODELS = Path('./models/'); MODELS.ls()

def inputs_same(x1,x2): return np.all(np.isclose(x1, x2, atol=1e-5))

### get input ready

In [8]:
img = open_image("cat224x224.jpg")

In [9]:
img.data.shape

torch.Size([3, 224, 224])

In [10]:
normalized_img = normalize(img.data, tensor(imagenet_stats[0]), tensor(imagenet_stats[1]))

In [11]:
# batch x width x height x channel
torch_input = normalized_img.data[None, ...].permute(0,3,2,1)
numpy_input = to_np(torch_input)

In [12]:
numpy_input.shape

(1, 224, 224, 3)

### keras output

In [13]:
arch_name = 'vgg16'

In [14]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

keras_model = VGG16(weights='imagenet')

img_path = 'cat224x224.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
numpy_input = preprocess_input(x)

keras_output = keras_model.predict(numpy_input)
print('Predicted:', decode_predictions(keras_output, top=3)[0])

Predicted: [('n02124075', 'Egyptian_cat', 0.30435446), ('n02123045', 'tabby', 0.2010841), ('n02123159', 'tiger_cat', 0.15228517)]


In [15]:
keras_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [16]:
keras_model.save(MODELS/f'{arch_name}.h5')

### Keras to TF

In [17]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.
    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [18]:
# set model.training=False
tf.keras.backend.set_learning_phase(0)

In [20]:
sess = tf.keras.backend.get_session()

In [21]:
# get graph definition
gd = sess.graph.as_graph_def()

# # fix batch norm nodes
# for node in graph_def.node:
#     if (node.op == 'RefSwitch') or (node.op == 'Switch'):
#         node.op = 'Switch'
#         for index in range(len(node.input)):
#             if 'moving_' in node.input[index]:
#                 node.input[index] = node.input[index] + '/read'
#     elif node.op == 'AssignSub':
#         node.op = 'Sub'
#         if 'use_locking' in node.attr: del node.attr['use_locking']
#     elif node.op == 'AssignAdd':
#         node.op = 'Add'
#         if 'use_locking' in node.attr: del node.attr['use_locking']
#     elif node.op == 'Assign':
#         node.op = 'Identity'
#         if 'use_locking' in node.attr: del node.attr['use_locking']
#         if 'validate_shape' in node.attr: del node.attr['validate_shape']
#         if len(node.input) == 2:
#             # input0: ref: Should be from a Variable node. May be uninitialized.
#             # input1: value: The value to be assigned to the variable.
#             node.input[0] = node.input[1]
#             del node.input[1]

In [26]:
from tensorflow import graph_util

In [30]:
keras_model.outputs

[<tf.Tensor 'predictions/Softmax:0' shape=(?, 1000) dtype=float32>]

In [33]:
keras_model.output.name.split(':')[0]

'predictions/Softmax'

In [43]:
# generate protobuf
converted_graph_def = graph_util.convert_variables_to_constants(sess, gd, ['predictions/Softmax'])
tf.train.write_graph(converted_graph_def, str(MODELS), f'{arch_name}.pb', as_text=False)

INFO:tensorflow:Froze 32 variables.


INFO:tensorflow:Froze 32 variables.


INFO:tensorflow:Converted 32 variables to const ops.


INFO:tensorflow:Converted 32 variables to const ops.


'models/vgg16.pb'

### Test Keras and TF

In [44]:
import tf2onnx
from tensorflow.python.framework import graph_util

In [45]:
graph_def = tf.GraphDef()
with tf.gfile.GFile(str(MODELS/f'{arch_name}.pb'), "rb") as f:
    graph_def.ParseFromString(f.read())    

In [46]:
with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_def, name="prefix")

In [55]:
with tf.Session(graph=graph) as sess:   
    input_node = graph.get_tensor_by_name("prefix/"+keras_model.input.name)
    output_node = graph.get_tensor_by_name("prefix/"+keras_model.output.name)
    feed_dict={input_node: numpy_input}
    tf_output = sess.run(output_node, feed_dict)    

In [57]:
# Keras with TF
if not inputs_same(keras_output[0], tf_output[0]):
    raise Exception("Keras and TF outputs are not same")

### TF to CoreML 

In [58]:
import tfcoreml as tf_converter
import onnxmltools

In [61]:
"prefix/"+keras_model.input.name

'prefix/input_2:0'

In [62]:
graph.get_tensor_by_name("prefix/"+keras_model.output.name)

<tf.Tensor 'prefix/predictions/Softmax:0' shape=(?, 1000) dtype=float32>

In [63]:
coreml_model = tf_converter.convert(tf_model_path=str(MODELS/f'{arch_name}.pb'),
                         mlmodel_path=MODELS/f'{arch_name}.mlmodel',
                         input_name_shape_dict={"prefix/"+keras_model.input.name:(1,224,224,3)},
                         output_feature_names=[keras_model.output.name],
                         add_custom_layers=True)


Loading the TF graph...
Graph Loaded.
Done.
Now finding ops in the TF graph that can be dropped for inference
Now starting translation to CoreML graph.
Automatic shape interpretation succeeded for input blob input_2:0
1/126: Analysing op name: predictions/bias ( type:  Const )
2/126: Analysing op name: predictions/BiasAdd/ReadVariableOp ( type:  Identity )
3/126: Analysing op name: predictions/kernel ( type:  Const )
4/126: Analysing op name: predictions/MatMul/ReadVariableOp ( type:  Identity )
5/126: Analysing op name: fc2/bias ( type:  Const )
6/126: Analysing op name: fc2/BiasAdd/ReadVariableOp ( type:  Identity )
7/126: Analysing op name: fc2/kernel ( type:  Const )
8/126: Analysing op name: fc2/MatMul/ReadVariableOp ( type:  Identity )
9/126: Analysing op name: fc1/bias ( type:  Const )
10/126: Analysing op name: fc1/BiasAdd/ReadVariableOp ( type:  Identity )
11/126: Analysing op name: fc1/kernel ( type:  Const )
12/126: Analysing op name: fc1/MatMul/ReadVariableOp ( type:  Iden

117/126: Analysing op name: flatten/Reshape ( type:  Reshape )
118/126: Analysing op name: fc1/MatMul ( type:  MatMul )
119/126: Analysing op name: fc1/BiasAdd ( type:  BiasAdd )
120/126: Analysing op name: fc1/Relu ( type:  Relu )
121/126: Analysing op name: fc2/MatMul ( type:  MatMul )
122/126: Analysing op name: fc2/BiasAdd ( type:  BiasAdd )
123/126: Analysing op name: fc2/Relu ( type:  Relu )
124/126: Analysing op name: predictions/MatMul ( type:  MatMul )
125/126: Analysing op name: predictions/BiasAdd ( type:  BiasAdd )
126/126: Analysing op name: predictions/Softmax ( type:  Softmax )
Translation to CoreML spec completed. Now compiling and saving the CoreML model.

 Core ML model generated. Saved at location: models/vgg16.mlmodel 

Core ML input(s): 
 [name: "input_2__0"
type {
  multiArrayType {
    shape: 3
    shape: 224
    shape: 224
    dataType: DOUBLE
  }
}
]
Core ML output(s): 
 [name: "predictions__Softmax__0"
type {
  multiArrayType {
    shape: 1000
    dataType: DO

### Test Keras and CoreML

### CoreML to ONNX

In [ ]:
# Save ONNX model
onnx_model = onnxmltools.convert_coreml(coreml_model, str(MODELS/f'{arch_name}.mlmodel'))
# onnxmltools.utils.save_text(onnx_model, MODELS/f'{arch_name}.json')

In [66]:
onnxmltools.utils.save_model(onnx_model,  str(MODELS/f'{arch_name}.onnx'))

### Test Keras and ONNX

### TF to TFlite

In [70]:
converter = tf.lite.TFLiteConverter.from_frozen_graph(str(MODELS/f'{arch_name}.pb'),
                                                      keras_model.input.name.split(":")[:1],
                                                      keras_model.output.name.split(":")[:1])
tflite_model = converter.convert()
open(MODELS/f"{arch_name}.tflite", "wb").write(tflite_model)

553436732

In [72]:
MODELS.ls()

[PosixPath('models/vgg16.onnx'),
 PosixPath('models/vgg16.tflite'),
 PosixPath('models/vgg16.mlmodel'),
 PosixPath('models/vgg16.pb'),
 PosixPath('models/vgg16.h5')]

### Test Keras and TFlite

In [74]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=str(MODELS/f"{arch_name}.tflite"))
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [76]:
input_details, output_details

([{'name': 'input_2',
   'index': 50,
   'shape': array([  1, 224, 224,   3], dtype=int32),
   'dtype': numpy.float32,
   'quantization': (0.0, 0)}],
 [{'name': 'predictions/Softmax',
   'index': 53,
   'shape': array([   1, 1000], dtype=int32),
   'dtype': numpy.float32,
   'quantization': (0.0, 0)}])

In [78]:
# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = numpy_input
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
tflite_output = interpreter.get_tensor(output_details[0]['index'])

In [79]:
tflite_output

array([[7.263924e-06, 4.362771e-05, 2.551505e-06, 2.809446e-06, ..., 8.950497e-06, 2.359440e-05, 2.745928e-04,
        4.154908e-04]], dtype=float32)

In [81]:
# Keras with TF
if not inputs_same(keras_output[0], tflite_output[0]):
    raise Exception("Keras and TFLite outputs are not same")

### Notes

- There is a problem during batchnorm conversion - https://www.bountysource.com/issues/36614355-unable-to-import-frozen-graph-with-batchnorm